### Задания

В самостоятельном проекте нужно проанализировать данные о фондах и инвестициях и написать запросы к базе. 

In [ ]:
#Задание1
#1.Посчитайте, сколько компаний закрылось.

SELECT COUNT(status)
FROM company
WHERE status LIKE 'closed'

In [ ]:
#Задание2
#2.Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC

In [ ]:
#Задание3
#Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM acquired_at) BETWEEN 2011 AND 2013 

In [ ]:
#Задание4
#Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

SELECT first_name,
       last_name,
       twitter_username    
FROM people       
WHERE twitter_username LIKE 'Silver%'

In [ ]:
#Задание5
#Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

SELECT *
FROM people
WHERE last_name LIKE 'K%'
AND twitter_username LIKE '%money%'

In [ ]:
#Задание6
#Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC

In [ ]:
#Задание7
#Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
#Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round 
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0
AND MIN(raised_amount) <> MAX(raised_amount)

In [ ]:
#Задание8
#Создайте поле с категориями:
# *Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
# *Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
# *Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
#Отобразите все поля таблицы fund и новое поле с категориями.

SELECT *,
        CASE
            WHEN invested_companies >= 100 THEN 'high_activity'
            WHEN invested_companies < 100 AND invested_companies >= 20 THEN 'middle_activity'
            WHEN invested_companies < 20 THEN 'low_activity'
        END
FROM fund 

In [ ]:
#Задание9
#Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds),0)
FROM fund
GROUP BY CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END
ORDER BY ROUND(AVG(investment_rounds),0);


In [ ]:
#Задание10
#Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
#Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.
#Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM founded_at) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) <> 0
ORDER BY AVG(invested_companies) DESC,
         country_code
LIMIT 10         

In [ ]:
#Задание11
#Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

SELECT 
      p.first_name,
      p.last_name,
      e.instituition 
FROM people AS p 
LEFT JOIN education AS e ON p.id = e.person_id

In [1]:
#Задание12
#Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

SELECT 
       c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
JOIN people AS p ON c.id = p.company_id
JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5            


In [ ]:
#Задание13
#Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

SELECT DISTINCT c.name
FROM funding_round AS fr
JOIN company AS c ON c.id = fr.company_id 
WHERE fr.is_first_round = 1 
AND fr.is_last_round = 1
AND c.status = 'closed'

In [ ]:
#Задание14
#Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN (
    SELECT c.id
    FROM funding_round AS fr
    JOIN company AS c ON c.id = fr.company_id 
    WHERE fr.is_first_round = 1 
    AND fr.is_last_round = 1
    AND c.status = 'closed'
)

In [ ]:
#Задание15
#Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

SELECT DISTINCT p.id,
       e.instituition
FROM people AS p
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (
    SELECT c.id
    FROM funding_round AS fr
    JOIN company AS c ON c.id = fr.company_id 
    WHERE fr.is_first_round = 1 
    AND fr.is_last_round = 1
    AND c.status = 'closed'
)

In [ ]:
#Задание16
#Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

SELECT DISTINCT p.id,
       COUNT(e.instituition)
FROM people AS p
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (
    SELECT c.id
    FROM funding_round AS fr
    JOIN company AS c ON c.id = fr.company_id 
    WHERE fr.is_first_round = 1 
    AND fr.is_last_round = 1
    AND c.status = 'closed'
)
GROUP BY p.id

In [2]:
#Задание17
#Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

SELECT AVG(table_educ.cnt)
FROM (
    SELECT p.id,
    COUNT(e.instituition)
    FROM people AS p
    JOIN education AS e ON p.id = e.person_id
    WHERE p.company_id IN (
        SELECT c.id
        FROM funding_round AS fr
        JOIN company AS c ON c.id = fr.company_id 
        WHERE fr.is_first_round = 1 
        AND fr.is_last_round = 1
        AND c.status = 'closed'
    )
    GROUP BY p.id
) AS table_educ(id, cnt)

In [ ]:
#Задание18
#Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
#*(сервис, запрещённый на территории РФ)

SELECT AVG(table_facebook.cnt)
FROM (SELECT e.person_id,
             COUNT(e.instituition)
      FROM education AS e
      WHERE e.person_id IN (
          SELECT p.id
          FROM people AS p 
          JOIN company AS c ON p.company_id = c.id
          WHERE c.name = 'Facebook'
      )
      GROUP BY e.person_id
     ) 
AS table_facebook(id, cnt)

In [ ]:
#Задание19
#Составьте таблицу из полей: name_of_fund — название фонда; name_of_company — название компании; amount — сумма инвестиций, которую привлекла компания в раунде.
#В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

SELECT f.name AS name_of_fund,
       c.name AS name_of_company ,
       fr.raised_amount AS amount 
FROM investment AS i
JOIN fund AS f ON i.fund_id = f.id
JOIN company AS c ON c.id = i.company_id
JOIN (SELECT * 
      FROM funding_round
     WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2012 AND 2013) AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6
GROUP BY f.name,
         c.name,
         fr.raised_amount

In [ ]:
#Задание20
#Выгрузите таблицу, в которой будут такие поля:
#*название компании-покупателя;
#*сумма сделки;
#*название компании, которую купили;
#*сумма инвестиций, вложенных в купленную компанию;
#*доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
#Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
#Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

SELECT c1.name AS acquiring_company,
       a.price_amount,
       c2.name AS acquired_company,
       c2.funding_total,
       ROUND(a.price_amount/c2.funding_total, 0)
FROM acquisition AS a 
JOIN company AS c1 ON a.acquiring_company_id = c1.id
JOIN company AS c2 ON a.acquired_company_id = c2.id
WHERE a.price_amount <> 0
AND c2.funding_total <> 0
ORDER BY price_amount DESC,
         acquired_company
LIMIT 10         


In [ ]:
#Задание21
#Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

SELECT c.name,
       fr.month 
FROM company AS c
JOIN (SELECT company_id,
      EXTRACT(MONTH FROM funded_at) 
      FROM funding_round
      WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
      AND raised_amount <> 0
     ) AS fr(id, month) ON c.id = fr.id
AND c.category_code = 'social'

In [ ]:
#Задание22
#Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
#*номер месяца, в котором проходили раунды;
#*количество уникальных названий фондов из США, которые инвестировали в этом месяце;
#*количество компаний, купленных за этот месяц;
#*общая сумма сделок по покупкам в этом месяце.

WITH 
fund_USA(month, cnt) AS (
             SELECT EXTRACT(MONTH FROM fr.funded_at),
                    COUNT(DISTINCT f.name)
             FROM fund AS f
             JOIN investment AS i ON f.id = i.fund_id
             JOIN funding_round AS fr ON i.funding_round_id = fr.id
             WHERE f.country_code = 'USA'
             AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
             GROUP BY EXTRACT(MONTH FROM fr.funded_at)
),

acq_comp(month, cnt, sum) AS (
             SELECT EXTRACT(MONTH FROM acquired_at),
                    COUNT(acquired_company_id),
                    SUM(price_amount)
             FROM acquisition
             WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
             GROUP BY EXTRACT(MONTH FROM acquired_at))

SELECT fund_USA.month,
       fund_USA.cnt,
       acq_comp.cnt,
       acq_comp.sum
FROM fund_USA
JOIN acq_comp ON fund_USA.month = acq_comp.month

In [ ]:
#Задание23
#Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

WITH 
table_2011(code, avg) AS (SELECT country_code,
                     AVG(funding_total) 
                     FROM company
                     WHERE EXTRACT(YEAR FROM founded_at) = 2011
                     GROUP BY country_code),

table_2012(code, avg) AS (SELECT country_code,
                     AVG(funding_total) 
                     FROM company
                     WHERE EXTRACT(YEAR FROM founded_at) = 2012
                     GROUP BY country_code),
                     
table_2013(code, avg) AS (SELECT country_code,
                     AVG(funding_total) 
                     FROM company
                     WHERE EXTRACT(YEAR FROM founded_at) = 2013
                     GROUP BY country_code)

SELECT table_2011.code,
       table_2011.avg,
       table_2012.avg,
       table_2013.avg
FROM table_2011
JOIN table_2012 ON table_2011.code = table_2012.code
JOIN table_2013 ON table_2011.code = table_2013.code
ORDER BY table_2011.avg DESC